In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv("env_waselee.tsv", sep='\t|,',engine='python')
data.head()

,waste,wst_oper,unit,geo\time,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
0,EE_ATD,COL,KG_HAB,AT,:,0.03,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0,0.02,0.01,0.01,0
1,EE_ATD,COL,KG_HAB,BE,:,0.04,0.07,0.06,0.24,0.21,0.17,0.15,0.12,0.19,0.03,0,:,:
2,EE_ATD,COL,KG_HAB,BG,:,0.03,0.02,0.02 d,0 d,0.02,0.04,0.03,0.03,0.01,:,:,:,:
3,EE_ATD,COL,KG_HAB,CY,:,0,0,0.08,0.01,0.01,0.02,0.01,0,0,0,0,:,:
4,EE_ATD,COL,KG_HAB,CZ,:,0.01,0.03,0.02,0,0,0.01,0.01,0.01,0,0,0,:,:


### Nettoyage des données

* Dans ce dataset les NaN sont défini par ':' (ex : ligne 0 , colonne 2018)

* Dans ce dataset, certaines données sont concaténées avec des "flags" qui ajoute un complément d'info inutile dans notre cas ( ex : ligne 2, colonne 2015)

In [3]:
years = ['2005','2006 ','2007 ','2008 ','2009 ','2010 ','2011 ','2012 ','2013 ','2014 ','2015 ','2016 ','2017 ','2018 ']
flags = ['b','c','d','e','f','n','p','r','s','u','z']

In [4]:
def cleanData(data, years, flags) :   
    for y in years :
        # Remplace les valeurs nulles par 0
        data[y] = data[y].str.replace(': ','0')
        data[y] = data[y].str.replace(':','0')
        # Enlève les flags inutiles pour rendre les données exploitable numériquement
        for f in flags :
            data.loc[data[y].str.contains(str(f)), y] = data[y].str.replace(f, '')      
    return data

def detectNaN(data,years):
    cpt = 0
    for y in years :
        if data[y].str.contains(':').any() :
            print(y +" pas néttoyé")
            cpt += 1
    if cpt == 0 :
        print('les données sont nettoyés')
    

In [5]:
data = cleanData(data,years,flags)

In [6]:
detectNaN(data,years)

les données sont nettoyés


### Conversion des données

In [7]:
def to_numeric(data,years): 
    for y in years :
        data[y] = data[y].apply(pd.to_numeric)

In [8]:
to_numeric(data,years)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10084 entries, 0 to 10083
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   waste     10084 non-null  object 
 1   wst_oper  10084 non-null  object 
 2   unit      10084 non-null  object 
 3   geo\time  10084 non-null  object 
 4   2018      10084 non-null  float64
 5   2017      10084 non-null  float64
 6   2016      10084 non-null  float64
 7   2015      10084 non-null  float64
 8   2014      10084 non-null  float64
 9   2013      10084 non-null  float64
 10  2012      10084 non-null  float64
 11  2011      10084 non-null  float64
 12  2010      10084 non-null  float64
 13  2009      10084 non-null  float64
 14  2008      10084 non-null  float64
 15  2007      10084 non-null  float64
 16  2006      10084 non-null  float64
 17  2005      10084 non-null  float64
dtypes: float64(14), object(4)
memory usage: 1.4+ MB


### Ajout colonne sum_year pour le cumul des années

In [10]:
data['sum_year'] = data.iloc[:,4:].sum(axis=1)
data.head()

,waste,wst_oper,unit,geo\time,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,sum_year
0,EE_ATD,COL,KG_HAB,AT,0.0,0.03,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.02,0.01,0.01,0.0,0.15
1,EE_ATD,COL,KG_HAB,BE,0.0,0.04,0.07,0.06,0.24,0.21,0.17,0.15,0.12,0.19,0.03,0.00,0.00,0.0,1.28
2,EE_ATD,COL,KG_HAB,BG,0.0,0.03,0.02,0.02,0.00,0.02,0.04,0.03,0.03,0.01,0.00,0.00,0.00,0.0,0.20
3,EE_ATD,COL,KG_HAB,CY,0.0,0.00,0.00,0.08,0.01,0.01,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.0,0.13
4,EE_ATD,COL,KG_HAB,CZ,0.0,0.01,0.03,0.02,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.0,0.09


### Suppression des lignes qui ne nous interesse pas

In [11]:
type_waste = ['EE_LHA','EE_SHA','EE_ITT','EE_CPV_CON','EE_TLS','TOTAL']
oper_waste = ['MKT','COL','RCY_REU','RCV','REU']
unit_waste = ['KG_HAB']

In [12]:
def deleteRowType(data,type_waste) :
    data = data[data['waste'].isin(type_waste)]
    return data

def deleteRowOper(data,oper_waste) :
    data = data[data['wst_oper'].isin(oper_waste)]
    return data

def deleteRowUnit(data,unit_waste) :
    data = data[data['unit'].isin(unit_waste)]
    return data

In [13]:
data = deleteRowType(data,type_waste)
data.shape

(4335, 19)

In [14]:
data['waste'].unique()

array(['EE_CPV_CON', 'EE_ITT', 'EE_LHA', 'EE_SHA', 'EE_TLS', 'TOTAL'],
      dtype=object)

In [15]:
data = deleteRowOper(data,oper_waste)
data.shape

(2143, 19)

In [16]:
data['wst_oper'].unique()

array(['COL', 'MKT', 'RCV', 'RCY_REU', 'REU'], dtype=object)

In [17]:
data = deleteRowUnit(data,unit_waste)
data.shape

(872, 19)

In [18]:
data['unit'].unique()

array(['KG_HAB'], dtype=object)

In [19]:
data.head()

,waste,wst_oper,unit,geo\time,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,sum_year
1540,EE_CPV_CON,COL,KG_HAB,AT,0.0,1.59,1.72,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.31
1541,EE_CPV_CON,COL,KG_HAB,BE,0.0,1.85,1.99,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.84
1542,EE_CPV_CON,COL,KG_HAB,CY,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1543,EE_CPV_CON,COL,KG_HAB,CZ,0.0,1.45,1.46,1.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.22
1544,EE_CPV_CON,COL,KG_HAB,DE,0.0,1.63,1.58,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.21


* On passe de 10 084 lignes à 872 

In [20]:
newcols=[(data.columns[3],"countryID")]+[(data.columns[i],"y"+data.columns[i][:4]) for i in range(4,len(data.columns)-1)]
newcols=dict(newcols)
print(newcols)
data=data.rename(columns=newcols)
data.head()

{'geo\\time': 'countryID', '2018 ': 'y2018', '2017 ': 'y2017', '2016 ': 'y2016', '2015 ': 'y2015', '2014 ': 'y2014', '2013 ': 'y2013', '2012 ': 'y2012', '2011 ': 'y2011', '2010 ': 'y2010', '2009 ': 'y2009', '2008 ': 'y2008', '2007 ': 'y2007', '2006 ': 'y2006', '2005': 'y2005'}


,waste,wst_oper,unit,countryID,y2018,y2017,y2016,y2015,y2014,y2013,y2012,y2011,y2010,y2009,y2008,y2007,y2006,y2005,sum_year
1540,EE_CPV_CON,COL,KG_HAB,AT,0.0,1.59,1.72,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.31
1541,EE_CPV_CON,COL,KG_HAB,BE,0.0,1.85,1.99,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.84
1542,EE_CPV_CON,COL,KG_HAB,CY,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1543,EE_CPV_CON,COL,KG_HAB,CZ,0.0,1.45,1.46,1.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.22
1544,EE_CPV_CON,COL,KG_HAB,DE,0.0,1.63,1.58,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.21


In [21]:
data["countryID"]=data["countryID"].replace("UK","GB")
data

,waste,wst_oper,unit,countryID,y2018,y2017,y2016,y2015,y2014,y2013,y2012,y2011,y2010,y2009,y2008,y2007,y2006,y2005,sum_year
1540,EE_CPV_CON,COL,KG_HAB,AT,0.0,1.59,1.72,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,3.31
1541,EE_CPV_CON,COL,KG_HAB,BE,0.0,1.85,1.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,3.84
1542,EE_CPV_CON,COL,KG_HAB,CY,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
1543,EE_CPV_CON,COL,KG_HAB,CZ,0.0,1.45,1.46,1.31,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,4.22
1544,EE_CPV_CON,COL,KG_HAB,DE,0.0,1.63,1.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,3.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9782,TOTAL,REU,KG_HAB,RO,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00
9783,TOTAL,REU,KG_HAB,SE,0.0,0.01,0.01,0.03,0.05,0.03,0.00,0.00,0.02,0.02,0.00,0.00,0.0,0.0,0.17
9784,TOTAL,REU,KG_HAB,SI,0.0,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.0,0.0,0.03
9785,TOTAL,REU,KG_HAB,SK,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.0,0.0,0.07


In [22]:
data["countryID"]=data["countryID"].replace("EL","GR")
data["countryID"].unique()

array(['AT', 'BE', 'CY', 'CZ', 'DE', 'DK', 'GR', 'ES', 'FR', 'IS', 'LI',
       'MT', 'NL', 'PT', 'RO', 'SK', 'GB', 'EE', 'BG', 'EU27_2020',
       'EU28', 'FI', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'NO', 'PL',
       'SE', 'SI'], dtype=object)

In [23]:
data.to_csv("cleaned_data.csv")

#### Annexe graph

In [24]:
annexe = pd.read_csv("env_waselee.tsv", sep='\t|,',engine='python')

In [25]:
df = pd.read_csv("env_waselee.tsv", sep='\t|,',engine='python')

In [26]:
df['unit'].unique()

array(['KG_HAB', 'T', 'PC', 'AVG_3Y'], dtype=object)

In [27]:
annexe = cleanData(annexe,years,flags)

In [28]:
to_numeric(annexe,years)

In [29]:
unit_waste_annexe = ['T']

In [30]:
annexe = deleteRowType(annexe,type_waste)
annexe = deleteRowOper(annexe,oper_waste)
annexe = deleteRowUnit(annexe,unit_waste_annexe)
annexe.shape

(878, 18)

In [31]:
annexe['unit'].unique()

array(['T'], dtype=object)

In [32]:
annexe.head()

,waste,wst_oper,unit,geo\time,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
1557,EE_CPV_CON,COL,T,AT,12228.0,13957.0,15004.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1558,EE_CPV_CON,COL,T,BE,22381.0,21032.0,22562.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1559,EE_CPV_CON,COL,T,CY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1560,EE_CPV_CON,COL,T,CZ,15366.0,15323.0,15439.0,13793.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1561,EE_CPV_CON,COL,T,DE,118136.0,135109.0,130144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
newcols=[(annexe.columns[i],"y"+annexe.columns[i][:4]) for i in range(4,len(data.columns)-1)]
newcols=dict(newcols)
annexe = annexe.rename(columns=newcols)

In [34]:
annexe.shape

(878, 18)

In [35]:
annexe['waste'].unique()

array(['EE_CPV_CON', 'EE_ITT', 'EE_LHA', 'EE_SHA', 'EE_TLS', 'TOTAL'],
      dtype=object)

In [36]:
annexe['wst_oper'].unique()

array(['COL', 'MKT', 'RCV', 'RCY_REU', 'REU'], dtype=object)

In [37]:
annexe['unit'].unique()

array(['T'], dtype=object)

In [38]:
annexe.head()

,waste,wst_oper,unit,geo\time,y2018,y2017,y2016,y2015,y2014,y2013,y2012,y2011,y2010,y2009,y2008,y2007,y2006,y2005
1557,EE_CPV_CON,COL,T,AT,12228.0,13957.0,15004.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1558,EE_CPV_CON,COL,T,BE,22381.0,21032.0,22562.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1559,EE_CPV_CON,COL,T,CY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1560,EE_CPV_CON,COL,T,CZ,15366.0,15323.0,15439.0,13793.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1561,EE_CPV_CON,COL,T,DE,118136.0,135109.0,130144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
annexe.to_csv("annexe.csv")